<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/83_isochrones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [1]:
%%capture
!pip install mapclassify osmnx

In [2]:
import folium
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, box

In [3]:
extent = box(-66.685827421, 45.936655091, -66.601967062, 45.997701154)
G = ox.graph_from_polygon(extent, network_type='walk')
for u, v, data in G.edges(data=True):
    data['travel_time'] = data['length'] / 0.85  # m/s

In [4]:
node, coord = list(G.nodes(data=True))[150]
isochrone_times = [20, 15, 10, 5]

isochrone_polys = []
for time in isochrone_times:
    subgraph = nx.ego_graph(
        G, node, radius=time * 60, distance='travel_time'
    )
    node_points = [
        Point((data['x'], data['y']))
        for node, data in subgraph.nodes(data=True)
    ]
    polygon = Polygon(gpd.GeoSeries(node_points).unary_union.convex_hull)
    isochrone_polys.append(polygon)

In [5]:
m = (
    gpd.GeoDataFrame(
        geometry=isochrone_polys,
        crs='EPSG:4326'
    )
    .assign(walking_time=isochrone_times)
    .explore(
        tiles='CartoDB positron',
        column='walking_time',
        cmap='viridis',
        style_kwds={'fillOpacity': 0.4}
    )
)

folium.Marker(location=(coord['y'], coord['x']), popup="Paris").add_to(m)

m

In [7]:
m.save('83_isochrones.html')